# Calibrated 1980 initial conditions

For a fair comparison we need the right ocean layer temperatures to initialise FaIR with in 1980. So we re-run the constrained calibrated ensemble *with AR6 best estimate forcing* and save out results in 1980.

In [ ]:
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
scenarios = ["AR6_hist"]  # doesn't matter which

In [ ]:
df_configs = pd.read_csv('../data/calibrated_constrained_parameters_v1.1.0.csv', index_col=0)
df_configs.index.name = 'config'

In [ ]:
f = FAIR()
f.define_time(1750, 1980, 1)
f.define_scenarios(scenarios)
f.define_configs(df_configs.index)
# Create an aggregate forcing time series specie.
f.define_species(['Fext'],
    {
        'Fext': {
            'type': 'unspecified',
            'input_mode': 'forcing',
            'greenhouse_gas': False,
            'aerosol_chemistry_from_emissions': False,
            'aerosol_chemistry_from_concentration': False
        }
    }
)
f.allocate()

In [ ]:
# climate response
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(f.climate_configs["ocean_heat_transfer"], df_configs.loc[:, "kappa1":"kappa3"].values)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], True)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

In [ ]:
df_ar6 = pd.read_csv('../data/AR6_ERF_1750-2019.csv', index_col=0)
df_ar6.loc[1750:1980, 'total']

In [ ]:
f.forcing[:, 0, :, 0] = df_ar6.loc[1750:1980, 'total'].values[:, None]

In [ ]:
# initial conditions: 1750
initialise(f.forcing, 0)
initialise(f.temperature, 0)

In [ ]:
f.run()

In [ ]:
# rebase to 1850-1900 means
weight_51yr = np.ones(52)
weight_51yr[0] = 0.5
weight_51yr[-1] = 0.5

In [ ]:
temp_rebased = f.temperature - np.average(f.temperature[100:152, ...], weights=weight_51yr, axis=0)

In [ ]:
temp_rebased[-1, 0, :, :]

In [ ]:
temp_rebased[-1, 0, :, :].to_pandas()

In [ ]:
temp_rebased[-1, 0, :, :].to_pandas().to_csv('../output/fair_1980_state.csv')